# Exploratory Data Analysis

This notebook conducts comprehensive exploratory data analysis on multiple datasets including:
- GDP per capita (World Bank)
- Public health expenditure as share of GDP
- Share of population that disagrees vaccines are effective
- Share of population that is urban

## Objectives
1. Load and examine each dataset
2. Understand data structure and quality
3. Identify missing values and data issues
4. Perform statistical summaries
5. Create visualizations
6. Explore relationships between variables


In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from pathlib import Path

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
warnings.filterwarnings('ignore')

# Set plotting style
try:
    plt.style.use('seaborn-v0_8-darkgrid')
except:
    try:
        plt.style.use('seaborn-darkgrid')
    except:
        plt.style.use('ggplot')
sns.set_palette("husl")

print("Libraries imported successfully!")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"Matplotlib version: {plt.matplotlib.__version__}")
print(f"Seaborn version: {sns.__version__}")

## 1. Data Loading


In [ ]:
# Load CSV files
data_dir = Path('.')

# Load GDP per capita data
gdp_df = pd.read_csv('gdp-per-capita-worldbank.csv')
print("GDP per capita data loaded")
print(f"Shape: {gdp_df.shape}")

# Load public health expenditure data
health_exp_df = pd.read_csv('public-health-expenditure-share-gdp.csv')
print("\nPublic health expenditure data loaded")
print(f"Shape: {health_exp_df.shape}")

# Load vaccine disagreement data
vaccine_df = pd.read_csv('share-disagrees-vaccines-are-effective.csv')
print("\nVaccine disagreement data loaded")
print(f"Shape: {vaccine_df.shape}")

# Load urban population data
urban_df = pd.read_csv('share-of-population-urban.csv')
print("\nUrban population data loaded")
print(f"Shape: {urban_df.shape}")

# Load measles reporting data (from Excel file 1)
excel1_df = pd.read_csv('measles-reporting-data.csv')
print("\nMeasles reporting data loaded")
print(f"Shape: {excel1_df.shape}")

# Load household size composition data (from Excel file 2)
excel2_df = pd.read_csv('undesa_pd_2022_hh-size-composition.csv')
print("\nHousehold size composition data loaded")
print(f"Shape: {excel2_df.shape}")
print(f"Shape: {excel2_df.shape}")

In [ ]:
# Define column names for easier reference
gdp_col = [col for col in gdp_df.columns if 'GDP' in col][0]
health_col = [col for col in health_exp_df.columns if 'health' in col.lower() or 'expenditure' in col.lower()][0]
vaccine_col = [col for col in vaccine_df.columns if 'vaccine' in col.lower() or 'disagree' in col.lower()][0]
urban_col = [col for col in urban_df.columns if 'urban' in col.lower()][0]

print("Column names identified:")
print(f"  GDP column: {gdp_col}")
print(f"  Health expenditure column: {health_col}")
print(f"  Vaccine disagreement column: {vaccine_col}")
print(f"  Urban population column: {urban_col}")


## 2. Dataset Overview


In [ ]:
# Display basic information about each dataset
datasets = {
    'GDP per capita': gdp_df,
    'Health Expenditure': health_exp_df,
    'Vaccine Disagreement': vaccine_df,
    'Urban Population': urban_df
}

for name, df in datasets.items():
    print(f"\n{'='*60}")
    print(f"{name} Dataset")
    print(f"{'='*60}")
    print(f"Shape: {df.shape}")
    print(f"\nColumn names:")
    print(df.columns.tolist())
    print(f"\nFirst few rows:")
    print(df.head())
    print(f"\nData types:")
    print(df.dtypes)
    print(f"\nBasic statistics:")
    print(df.describe())


## 3. Data Quality Assessment


In [ ]:
# Check for missing values in each dataset
for name, df in datasets.items():
    print(f"\n{'='*60}")
    print(f"{name} - Missing Values")
    print(f"{'='*60}")
    missing = df.isnull().sum()
    missing_pct = (missing / len(df)) * 100
    missing_df = pd.DataFrame({
        'Missing Count': missing,
        'Missing Percentage': missing_pct
    })
    print(missing_df[missing_df['Missing Count'] > 0])
    if missing_df['Missing Count'].sum() == 0:
        print("No missing values found!")
    
    # Check for duplicates
    duplicates = df.duplicated().sum()
    print(f"\nDuplicate rows: {duplicates}")


## 4. Temporal Analysis


In [ ]:
# Analyze temporal coverage for each dataset
for name, df in datasets.items():
    if 'Year' in df.columns:
        print(f"\n{name} - Year Range:")
        print(f"  Min Year: {df['Year'].min()}")
        print(f"  Max Year: {df['Year'].max()}")
        print(f"  Year Range: {df['Year'].max() - df['Year'].min()} years")
        print(f"  Unique Years: {df['Year'].nunique()}")
        print(f"  Unique Countries: {df['Entity'].nunique() if 'Entity' in df.columns else 'N/A'}")
        
        # Year distribution
        year_counts = df['Year'].value_counts().sort_index()
        print(f"\n  Years with most data points:")
        print(year_counts.head(10))


## 5. Geographic Coverage Analysis


In [ ]:
# Analyze geographic coverage
for name, df in datasets.items():
    if 'Entity' in df.columns:
        print(f"\n{name} - Geographic Coverage:")
        print(f"  Total unique countries/entities: {df['Entity'].nunique()}")
        print(f"\n  Sample of countries:")
        print(df['Entity'].unique()[:20])
        
        if 'World regions according to OWID' in df.columns:
            print(f"\n  World regions:")
            regions = df['World regions according to OWID'].value_counts()
            print(regions)


## 6. Visualizations - GDP per Capita


In [ ]:
# Clean column name for GDP
gdp_col = [col for col in gdp_df.columns if 'GDP' in col][0]

# Plot 1: GDP per capita distribution
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Distribution of GDP per capita
axes[0, 0].hist(gdp_df[gdp_col].dropna(), bins=50, edgecolor='black', alpha=0.7)
axes[0, 0].set_title('Distribution of GDP per Capita', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('GDP per Capita (PPP, constant 2021 international $)')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].grid(True, alpha=0.3)

# Log scale distribution
axes[0, 1].hist(np.log10(gdp_df[gdp_col].dropna()), bins=50, edgecolor='black', alpha=0.7)
axes[0, 1].set_title('Distribution of GDP per Capita (Log Scale)', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Log10(GDP per Capita)')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].grid(True, alpha=0.3)

# GDP over time (average across all countries)
gdp_by_year = gdp_df.groupby('Year')[gdp_col].mean()
axes[1, 0].plot(gdp_by_year.index, gdp_by_year.values, linewidth=2, marker='o')
axes[1, 0].set_title('Average GDP per Capita Over Time', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Year')
axes[1, 0].set_ylabel('Average GDP per Capita')
axes[1, 0].grid(True, alpha=0.3)

# Top 10 countries by latest GDP
latest_year = gdp_df['Year'].max()
latest_gdp = gdp_df[gdp_df['Year'] == latest_year].nlargest(10, gdp_col)
axes[1, 1].barh(latest_gdp['Entity'], latest_gdp[gdp_col])
axes[1, 1].set_title(f'Top 10 Countries by GDP per Capita ({latest_year})', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('GDP per Capita')
axes[1, 1].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()


## 7. Visualizations - Health Expenditure


In [ ]:
# Get health expenditure column name
health_col = [col for col in health_exp_df.columns if 'health' in col.lower() or 'expenditure' in col.lower()][0]

# Plot health expenditure analysis
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Distribution
axes[0, 0].hist(health_exp_df[health_col].dropna(), bins=50, edgecolor='black', alpha=0.7)
axes[0, 0].set_title('Distribution of Health Expenditure (% of GDP)', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Health Expenditure (% of GDP)')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].grid(True, alpha=0.3)

# Over time
health_by_year = health_exp_df.groupby('Year')[health_col].mean()
axes[0, 1].plot(health_by_year.index, health_by_year.values, linewidth=2, marker='o', color='green')
axes[0, 1].set_title('Average Health Expenditure Over Time', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Year')
axes[0, 1].set_ylabel('Average Health Expenditure (% of GDP)')
axes[0, 1].grid(True, alpha=0.3)

# Top countries by latest health expenditure
latest_year_health = health_exp_df['Year'].max()
latest_health = health_exp_df[health_exp_df['Year'] == latest_year_health].nlargest(10, health_col)
axes[1, 0].barh(latest_health['Entity'], latest_health[health_col], color='green')
axes[1, 0].set_title(f'Top 10 Countries by Health Expenditure ({latest_year_health})', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Health Expenditure (% of GDP)')
axes[1, 0].grid(True, alpha=0.3, axis='x')

# Box plot by decade
health_exp_df['Decade'] = (health_exp_df['Year'] // 10) * 10
recent_data = health_exp_df[health_exp_df['Decade'] >= 2000]
axes[1, 1].boxplot([recent_data[recent_data['Decade'] == d][health_col].dropna() 
                    for d in sorted(recent_data['Decade'].unique())],
                   labels=sorted(recent_data['Decade'].unique()))
axes[1, 1].set_title('Health Expenditure Distribution by Decade', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Decade')
axes[1, 1].set_ylabel('Health Expenditure (% of GDP)')
axes[1, 1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()


## 8. Visualizations - Vaccine Disagreement


In [ ]:
# Get vaccine column name
vaccine_col = [col for col in vaccine_df.columns if 'vaccine' in col.lower() or 'disagree' in col.lower()][0]

# Plot vaccine disagreement analysis
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Distribution
axes[0, 0].hist(vaccine_df[vaccine_col].dropna(), bins=30, edgecolor='black', alpha=0.7, color='orange')
axes[0, 0].set_title('Distribution of Vaccine Disagreement (%)', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Share that Disagrees Vaccines are Effective (%)')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].grid(True, alpha=0.3)

# Over time
vaccine_by_year = vaccine_df.groupby('Year')[vaccine_col].mean()
axes[0, 1].plot(vaccine_by_year.index, vaccine_by_year.values, linewidth=2, marker='o', color='orange')
axes[0, 1].set_title('Average Vaccine Disagreement Over Time', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Year')
axes[0, 1].set_ylabel('Average Disagreement (%)')
axes[0, 1].grid(True, alpha=0.3)

# Countries with highest disagreement (latest year)
latest_year_vaccine = 2024
latest_vaccine = vaccine_df[vaccine_df['Year'] == latest_year_vaccine].nlargest(10, vaccine_col)
axes[1, 0].barh(latest_vaccine['Entity'], latest_vaccine[vaccine_col], color='orange')
axes[1, 0].set_title(f'Top 10 Countries by Vaccine Disagreement ({latest_year_vaccine})', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Disagreement (%)')
axes[1, 0].grid(True, alpha=0.3, axis='x')

# Countries with lowest disagreement
lowest_vaccine = vaccine_df[vaccine_df['Year'] == latest_year_vaccine].nsmallest(10, vaccine_col)
axes[1, 1].barh(lowest_vaccine['Entity'], lowest_vaccine[vaccine_col], color='lightgreen')
axes[1, 1].set_title(f'Top 10 Countries with Lowest Vaccine Disagreement ({latest_year_vaccine})', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Disagreement (%)')
axes[1, 1].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()


## 9. Visualizations - Urban Population


## 10. Visualizations - Measles Reporting Data


In [ ]:
# Load measles data if not already loaded
if 'excel1_df' not in locals():
    excel1_df = pd.read_csv('measles-reporting-data.csv')

# Clean column names (remove BOM if present)
excel1_df.columns = excel1_df.columns.str.replace('\ufeff', '')

# Display basic info
print("Measles Reporting Data Overview:")
print(f"Shape: {excel1_df.shape}")
print(f"\nColumns: {excel1_df.columns.tolist()}")
print(f"\nFirst few rows:")
print(excel1_df.head())
print(f"\nData types:")
print(excel1_df.dtypes)
print(f"\nMissing values:")
print(excel1_df.isnull().sum())


In [ ]:
# Visualize measles reporting data
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# Get column names
cases_col = 'Total confirmed  measles cases'
incidence_col = 'Measles incidence rate per 1\'000\'000  total population'
lab_col = 'Lab confirmed'
pop_col = 'Total population'

# 1. Distribution of total confirmed cases
axes[0, 0].hist(excel1_df[cases_col].dropna(), bins=50, edgecolor='black', alpha=0.7, color='red')
axes[0, 0].set_title('Distribution of Total Confirmed Measles Cases', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Total Confirmed Cases')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].set_yscale('log')
axes[0, 0].grid(True, alpha=0.3)

# 2. Measles cases over time (global)
cases_by_year = excel1_df.groupby('Year')[cases_col].sum()
axes[0, 1].plot(cases_by_year.index, cases_by_year.values, linewidth=2, marker='o', color='red')
axes[0, 1].set_title('Global Measles Cases Over Time', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Year')
axes[0, 1].set_ylabel('Total Cases')
axes[0, 1].grid(True, alpha=0.3)

# 3. Incidence rate over time (average)
incidence_by_year = excel1_df.groupby('Year')[incidence_col].mean()
axes[0, 2].plot(incidence_by_year.index, incidence_by_year.values, linewidth=2, marker='o', color='orange')
axes[0, 2].set_title('Average Measles Incidence Rate Over Time', fontsize=14, fontweight='bold')
axes[0, 2].set_xlabel('Year')
axes[0, 2].set_ylabel('Incidence Rate (per 1M population)')
axes[0, 2].grid(True, alpha=0.3)

# 4. Top 10 countries by total cases (latest year)
latest_year_measles = excel1_df['Year'].max()
latest_measles = excel1_df[excel1_df['Year'] == latest_year_measles].nlargest(10, cases_col)
axes[1, 0].barh(latest_measles['Member State'], latest_measles[cases_col], color='red')
axes[1, 0].set_title(f'Top 10 Countries by Measles Cases ({latest_year_measles})', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Total Cases')
axes[1, 0].grid(True, alpha=0.3, axis='x')

# 5. Top 10 countries by incidence rate (latest year)
latest_incidence = excel1_df[excel1_df['Year'] == latest_year_measles].nlargest(10, incidence_col)
axes[1, 1].barh(latest_incidence['Member State'], latest_incidence[incidence_col], color='orange')
axes[1, 1].set_title(f'Top 10 Countries by Incidence Rate ({latest_year_measles})', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Incidence Rate (per 1M)')
axes[1, 1].grid(True, alpha=0.3, axis='x')

# 6. Cases by region (latest year)
region_cases = excel1_df[excel1_df['Year'] == latest_year_measles].groupby('Region')[cases_col].sum().sort_values(ascending=False)
axes[1, 2].bar(range(len(region_cases)), region_cases.values, color='crimson')
axes[1, 2].set_xticks(range(len(region_cases)))
axes[1, 2].set_xticklabels(region_cases.index, rotation=45, ha='right')
axes[1, 2].set_title(f'Measles Cases by Region ({latest_year_measles})', fontsize=14, fontweight='bold')
axes[1, 2].set_ylabel('Total Cases')
axes[1, 2].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()


In [ ]:
# Additional measles analysis: Confirmation types breakdown
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Lab confirmed vs other types over time
lab_by_year = excel1_df.groupby('Year')[lab_col].sum()
epi_by_year = excel1_df.groupby('Year')['Epidemiologically linked'].sum()
clinical_by_year = excel1_df.groupby('Year')['Clinically compatible'].sum()

axes[0].plot(lab_by_year.index, lab_by_year.values, label='Lab Confirmed', linewidth=2, marker='o')
axes[0].plot(epi_by_year.index, epi_by_year.values, label='Epidemiologically Linked', linewidth=2, marker='s')
axes[0].plot(clinical_by_year.index, clinical_by_year.values, label='Clinically Compatible', linewidth=2, marker='^')
axes[0].set_title('Measles Cases by Confirmation Type Over Time', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Year')
axes[0].set_ylabel('Number of Cases')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Heatmap: Cases by region and year (recent years)
recent_years = sorted(excel1_df['Year'].unique())[-10:]  # Last 10 years
region_year = excel1_df[excel1_df['Year'].isin(recent_years)].pivot_table(
    values=cases_col, index='Region', columns='Year', aggfunc='sum', fill_value=0
)
sns.heatmap(region_year, annot=True, fmt='.0f', cmap='YlOrRd', ax=axes[1], cbar_kws={'label': 'Total Cases'})
axes[1].set_title('Measles Cases Heatmap: Region vs Year', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Year')
axes[1].set_ylabel('Region')

plt.tight_layout()
plt.show()


## 11. Visualizations - Household Size Composition


In [ ]:
# Load household size data if not already loaded
if 'excel2_df' not in locals():
    excel2_df = pd.read_csv('undesa_pd_2022_hh-size-composition.csv')

# Clean column names (remove BOM if present)
excel2_df.columns = excel2_df.columns.str.replace('\ufeff', '')

# Remove empty columns
excel2_df = excel2_df.loc[:, ~excel2_df.columns.str.contains('^Unnamed')]
excel2_df = excel2_df.dropna(axis=1, how='all')

# Parse reference date to extract year
def extract_year(date_str):
    try:
        if pd.isna(date_str):
            return None
        # Handle different date formats: "1/7/10", "19/10/2015", etc.
        parts = str(date_str).split('/')
        if len(parts) == 3:
            year = parts[2]
            if len(year) == 2:
                # Convert 2-digit year to 4-digit (assuming 2000s)
                year = '20' + year if int(year) < 50 else '19' + year
            return int(year)
    except:
        return None

excel2_df['Year'] = excel2_df['Reference date (dd/mm/yyyy)'].apply(extract_year)

# Get household size column
hh_size_col = 'Average household size (number of members)'

# Convert household size to numeric, handling non-numeric values (like "..")
excel2_df[hh_size_col] = pd.to_numeric(excel2_df[hh_size_col], errors='coerce')

# Display basic info
print("Household Size Composition Data Overview:")
print(f"Shape: {excel2_df.shape}")
print(f"\nColumns: {excel2_df.columns.tolist()}")
print(f"\nFirst few rows:")
print(excel2_df.head())
print(f"\nData types:")
print(excel2_df.dtypes)
print(f"\nMissing values:")
print(excel2_df.isnull().sum())
print(f"\nUnique countries: {excel2_df['Country or area'].nunique()}")
print(f"\nData source categories: {excel2_df['Data source category'].value_counts()}")
print(f"\nHousehold size statistics:")
print(excel2_df[hh_size_col].describe())


In [ ]:
# Visualize household size composition data
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# Filter out invalid household sizes
valid_data = excel2_df[excel2_df[hh_size_col].notna() & (excel2_df[hh_size_col] > 0) & (excel2_df[hh_size_col] < 20)]

# 1. Distribution of household sizes
axes[0, 0].hist(valid_data[hh_size_col], bins=40, edgecolor='black', alpha=0.7, color='teal')
axes[0, 0].set_title('Distribution of Average Household Size', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Average Household Size (members)')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].grid(True, alpha=0.3)

# 2. Household size over time (global average)
if valid_data['Year'].notna().sum() > 0:
    hh_by_year = valid_data.groupby('Year')[hh_size_col].mean()
    axes[0, 1].plot(hh_by_year.index, hh_by_year.values, linewidth=2, marker='o', color='teal')
    axes[0, 1].set_title('Global Average Household Size Over Time', fontsize=14, fontweight='bold')
    axes[0, 1].set_xlabel('Year')
    axes[0, 1].set_ylabel('Average Household Size')
    axes[0, 1].grid(True, alpha=0.3)

# 3. Household size by data source
source_means = valid_data.groupby('Data source category')[hh_size_col].mean().sort_values(ascending=False)
axes[0, 2].barh(range(len(source_means)), source_means.values, color='teal')
axes[0, 2].set_yticks(range(len(source_means)))
axes[0, 2].set_yticklabels(source_means.index)
axes[0, 2].set_title('Average Household Size by Data Source', fontsize=14, fontweight='bold')
axes[0, 2].set_xlabel('Average Household Size')
axes[0, 2].grid(True, alpha=0.3, axis='x')

# 4. Countries with largest household sizes (most recent data)
if valid_data['Year'].notna().sum() > 0:
    latest_year_hh = valid_data['Year'].max()
    latest_hh_data = valid_data[valid_data['Year'] == latest_year_hh]
    # Get most recent entry per country
    latest_hh_data = latest_hh_data.sort_values('Year').drop_duplicates(subset='Country or area', keep='last')
    largest_hh = latest_hh_data.nlargest(10, hh_size_col)
    axes[1, 0].barh(largest_hh['Country or area'], largest_hh[hh_size_col], color='teal')
    axes[1, 0].set_title(f'Top 10 Countries by Household Size ({latest_year_hh})', fontsize=14, fontweight='bold')
    axes[1, 0].set_xlabel('Average Household Size')
    axes[1, 0].grid(True, alpha=0.3, axis='x')

# 5. Countries with smallest household sizes
if valid_data['Year'].notna().sum() > 0:
    smallest_hh = latest_hh_data.nsmallest(10, hh_size_col)
    axes[1, 1].barh(smallest_hh['Country or area'], smallest_hh[hh_size_col], color='lightblue')
    axes[1, 1].set_title(f'Top 10 Countries with Smallest Households ({latest_year_hh})', fontsize=14, fontweight='bold')
    axes[1, 1].set_xlabel('Average Household Size')
    axes[1, 1].grid(True, alpha=0.3, axis='x')

# 6. Box plot by data source
source_data = [valid_data[valid_data['Data source category'] == source][hh_size_col].dropna() 
               for source in valid_data['Data source category'].value_counts().head(5).index]
axes[1, 2].boxplot(source_data, labels=valid_data['Data source category'].value_counts().head(5).index)
axes[1, 2].set_title('Household Size Distribution by Data Source', fontsize=14, fontweight='bold')
axes[1, 2].set_ylabel('Average Household Size')
axes[1, 2].tick_params(axis='x', rotation=45)
axes[1, 2].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()


In [ ]:
# Additional household size analysis: Trends for selected countries
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Get countries with multiple data points over time
country_years = valid_data.groupby('Country or area')['Year'].count()
countries_with_history = country_years[country_years >= 5].index[:10]  # Top 10 countries with most data points

# Plot trends for selected countries
for country in countries_with_history[:5]:
    country_data = valid_data[valid_data['Country or area'] == country].sort_values('Year')
    axes[0].plot(country_data['Year'], country_data[hh_size_col], marker='o', label=country, linewidth=2)

axes[0].set_title('Household Size Trends: Selected Countries', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Year')
axes[0].set_ylabel('Average Household Size')
axes[0].legend(loc='best', fontsize=8)
axes[0].grid(True, alpha=0.3)

# Distribution comparison: recent vs older data
if valid_data['Year'].notna().sum() > 0:
    recent_cutoff = valid_data['Year'].quantile(0.75)
    recent_data = valid_data[valid_data['Year'] >= recent_cutoff][hh_size_col]
    older_data = valid_data[valid_data['Year'] < recent_cutoff][hh_size_col]
    
    axes[1].hist(older_data.dropna(), bins=30, alpha=0.6, label=f'Before {int(recent_cutoff)}', color='lightcoral', edgecolor='black')
    axes[1].hist(recent_data.dropna(), bins=30, alpha=0.6, label=f'{int(recent_cutoff)} and later', color='teal', edgecolor='black')
    axes[1].set_title('Household Size Distribution: Historical Comparison', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('Average Household Size')
    axes[1].set_ylabel('Frequency')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


## 12. Relationship Analysis


In [ ]:
# Get urban population column name
urban_col = [col for col in urban_df.columns if 'urban' in col.lower()][0]

# Plot urban population analysis
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Distribution
axes[0, 0].hist(urban_df[urban_col].dropna(), bins=50, edgecolor='black', alpha=0.7, color='purple')
axes[0, 0].set_title('Distribution of Urban Population (%)', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Urban Population (% of total)')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].grid(True, alpha=0.3)

# Global trend over time
urban_by_year = urban_df.groupby('Year')[urban_col].mean()
axes[0, 1].plot(urban_by_year.index, urban_by_year.values, linewidth=2, marker='o', color='purple')
axes[0, 1].set_title('Average Urban Population Over Time', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Year')
axes[0, 1].set_ylabel('Average Urban Population (%)')
axes[0, 1].grid(True, alpha=0.3)

# Most urbanized countries (latest year)
latest_year_urban = urban_df['Year'].max()
latest_urban = urban_df[urban_df['Year'] == latest_year_urban].nlargest(10, urban_col)
axes[1, 0].barh(latest_urban['Entity'], latest_urban[urban_col], color='purple')
axes[1, 0].set_title(f'Top 10 Most Urbanized Countries ({latest_year_urban})', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Urban Population (%)')
axes[1, 0].grid(True, alpha=0.3, axis='x')

# Least urbanized countries
lowest_urban = urban_df[urban_df['Year'] == latest_year_urban].nsmallest(10, urban_col)
axes[1, 1].barh(lowest_urban['Entity'], lowest_urban[urban_col], color='brown')
axes[1, 1].set_title(f'Top 10 Least Urbanized Countries ({latest_year_urban})', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Urban Population (%)')
axes[1, 1].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()


In [ ]:
# Merge datasets to explore relationships
# Find a year with good coverage across all datasets

# Find common years across datasets
common_years = set(gdp_df['Year'].unique())
common_years = common_years.intersection(set(health_exp_df['Year'].unique()))
common_years = common_years.intersection(set(vaccine_df['Year'].unique()))
common_years = common_years.intersection(set(urban_df['Year'].unique()))

print(f"Common years across all datasets: {sorted(common_years)}")

# Find the year with the most countries having data in all datasets
best_year = None
max_countries = 0

for year in sorted(common_years, reverse=True):
    gdp_countries = set(gdp_df[gdp_df['Year'] == year]['Code'].unique())
    health_countries = set(health_exp_df[health_exp_df['Year'] == year]['Code'].unique())
    vaccine_countries = set(vaccine_df[vaccine_df['Year'] == year]['Code'].unique())
    urban_countries = set(urban_df[urban_df['Year'] == year]['Code'].unique())
    
    common_countries = gdp_countries & health_countries & vaccine_countries & urban_countries
    if len(common_countries) > max_countries:
        max_countries = len(common_countries)
        best_year = year

# Use the year with best coverage, or fall back to 2019/2020 if available
if best_year and max_countries >= 10:
    analysis_year = best_year
    print(f"\nBest year for analysis: {analysis_year} ({max_countries} countries with complete data)")
else:
    # Try 2019 or 2020 as they typically have good coverage
    for candidate_year in [2019, 2020, 2018]:
        if candidate_year in common_years:
            analysis_year = candidate_year
            gdp_countries = set(gdp_df[gdp_df['Year'] == analysis_year]['Code'].unique())
            health_countries = set(health_exp_df[health_exp_df['Year'] == analysis_year]['Code'].unique())
            vaccine_countries = set(vaccine_df[vaccine_df['Year'] == analysis_year]['Code'].unique())
            urban_countries = set(urban_df[urban_df['Year'] == analysis_year]['Code'].unique())
            common_countries = gdp_countries & health_countries & vaccine_countries & urban_countries
            print(f"\nUsing year {analysis_year} for relationship analysis ({len(common_countries)} countries with complete data)")
            break
    else:
        analysis_year = max(common_years)
        print(f"\nUsing most recent common year {analysis_year} for relationship analysis")


In [ ]:
# Merge datasets for the analysis year using inner join to only keep countries with all data
merged_df = gdp_df[gdp_df['Year'] == analysis_year][['Entity', 'Code', gdp_col]].copy()
merged_df = merged_df.merge(
    health_exp_df[health_exp_df['Year'] == analysis_year][['Code', health_col]], 
    on='Code', how='inner'  # Changed to inner to only keep countries with health data
)
merged_df = merged_df.merge(
    vaccine_df[vaccine_df['Year'] == analysis_year][['Code', vaccine_col]], 
    on='Code', how='inner'  # Changed to inner to only keep countries with vaccine data
)
merged_df = merged_df.merge(
    urban_df[urban_df['Year'] == analysis_year][['Code', urban_col]], 
    on='Code', how='inner'  # Changed to inner to only keep countries with urban data
)

print(f"Merged dataset shape: {merged_df.shape}")
print(f"\nMerged dataset columns: {merged_df.columns.tolist()}")
print(f"\nNumber of countries with complete data: {len(merged_df)}")
print(f"\nFirst few rows:")
print(merged_df.head(10))
print(f"\nMissing values in merged dataset:")
print(merged_df.isnull().sum())
print(f"\nData ranges:")
print(f"  GDP per capita: ${merged_df[gdp_col].min():,.2f} - ${merged_df[gdp_col].max():,.2f}")
print(f"  Health expenditure: {merged_df[health_col].min():.2f}% - {merged_df[health_col].max():.2f}%")
print(f"  Vaccine disagreement: {merged_df[vaccine_col].min():.2f}% - {merged_df[vaccine_col].max():.2f}%")
print(f"  Urban population: {merged_df[urban_col].min():.2f}% - {merged_df[urban_col].max():.2f}%")


In [ ]:
# Create correlation matrix for numeric variables
numeric_cols = [gdp_col, health_col, vaccine_col, urban_col]
correlation_df = merged_df[numeric_cols].corr()

# Plot correlation heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_df, annot=True, fmt='.2f', cmap='coolwarm', center=0,
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Correlation Matrix Between Variables', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

print("\nCorrelation Matrix:")
print(correlation_df)


## 12.5. Correlation Analysis: Metrics vs Measles Cases

This section analyzes the Spearman rank correlation between various socioeconomic and health metrics and measles incidence rate to identify potential relationships. Spearman correlation is used as it's more robust to outliers (few countries may dominate cases/incidence).


In [ ]:
# Prepare measles data for correlation analysis
# Load measles data if not already loaded
if 'excel1_df' not in locals():
    excel1_df = pd.read_csv('measles-reporting-data.csv')
    excel1_df.columns = excel1_df.columns.str.replace('\ufeff', '')

# Get measles cases column
cases_col = 'Total confirmed  measles cases'
incidence_col = 'Measles incidence rate per 1\'000\'000  total population'

# Find a year with good coverage for measles data
measles_years = excel1_df['Year'].unique()
print(f"Available years in measles data: {sorted(measles_years)}")

# Find year with most countries reporting measles data
year_counts = excel1_df.groupby('Year')['ISO country code'].nunique().sort_values(ascending=False)
best_measles_year = year_counts.index[0]
print(f"\nYear with most measles data: {best_measles_year} ({year_counts.iloc[0]} countries)")

# Use the same analysis_year from relationship analysis, or find best overlap
# Try to use a year that has both measles data and other metrics
overlap_years = set(measles_years).intersection(set(gdp_df['Year'].unique()))
overlap_years = overlap_years.intersection(set(health_exp_df['Year'].unique()))
overlap_years = overlap_years.intersection(set(urban_df['Year'].unique()))

if overlap_years:
    # Find year with most complete data
    correlation_year = None
    max_countries = 0
    for year in sorted(overlap_years, reverse=True):
        measles_codes = set(excel1_df[excel1_df['Year'] == year]['ISO country code'].dropna().unique())
        gdp_codes = set(gdp_df[gdp_df['Year'] == year]['Code'].unique())
        health_codes = set(health_exp_df[health_exp_df['Year'] == year]['Code'].unique())
        urban_codes = set(urban_df[urban_df['Year'] == year]['Code'].unique())
        
        common_codes = measles_codes & gdp_codes & health_codes & urban_codes
        if len(common_codes) > max_countries:
            max_countries = len(common_codes)
            correlation_year = year
    
    if correlation_year:
        print(f"\nUsing year {correlation_year} for correlation analysis ({max_countries} countries)")
    else:
        correlation_year = max(overlap_years)
        print(f"\nUsing year {correlation_year} for correlation analysis")
else:
    # Fallback to best available year
    correlation_year = best_measles_year
    print(f"\nUsing year {correlation_year} for correlation analysis (limited overlap)")


In [ ]:
# Merge measles data with other metrics for correlation analysis
correlation_df = excel1_df[excel1_df['Year'] == correlation_year][['ISO country code', cases_col, incidence_col]].copy()
correlation_df = correlation_df.rename(columns={'ISO country code': 'Code'})

# Merge with GDP
correlation_df = correlation_df.merge(
    gdp_df[gdp_df['Year'] == correlation_year][['Code', gdp_col]], 
    on='Code', how='inner'
)

# Merge with Health Expenditure
correlation_df = correlation_df.merge(
    health_exp_df[health_exp_df['Year'] == correlation_year][['Code', health_col]], 
    on='Code', how='inner'
)

# Merge with Urban Population
correlation_df = correlation_df.merge(
    urban_df[urban_df['Year'] == correlation_year][['Code', urban_col]], 
    on='Code', how='inner'
)

# Try to merge with Vaccine Disagreement if available for this year
if correlation_year in vaccine_df['Year'].values:
    correlation_df = correlation_df.merge(
        vaccine_df[vaccine_df['Year'] == correlation_year][['Code', vaccine_col]], 
        on='Code', how='left'  # Left join since vaccine data may be sparse
    )
else:
    print(f"Note: Vaccine disagreement data not available for year {correlation_year}")

# Try to merge with Household Size if available (use most recent year per country)
# Using country_converter to handle ISO code conversions
import country_converter as coco
cc = coco.CountryConverter()

# Define hh_size_col if not already defined
if 'hh_size_col' not in globals():
    hh_size_col = 'Average household size (number of members)'

if 'excel2_df' not in locals():
    excel2_df = pd.read_csv('undesa_pd_2022_hh-size-composition.csv')
    excel2_df.columns = excel2_df.columns.str.replace('\ufeff', '')
    excel2_df = excel2_df.loc[:, ~excel2_df.columns.str.contains('^Unnamed')]
    excel2_df[hh_size_col] = pd.to_numeric(excel2_df[hh_size_col], errors='coerce')
    
    # Define extract_year function if not already defined
    if 'extract_year' not in globals():
        def extract_year(date_str):
            try:
                if pd.isna(date_str):
                    return None
                parts = str(date_str).split('/')
                if len(parts) == 3:
                    year = parts[2]
                    if len(year) == 2:
                        year = '20' + year if int(year) < 50 else '19' + year
                    return int(year)
            except:
                return None
    
    excel2_df['Year'] = excel2_df['Reference date (dd/mm/yyyy)'].apply(extract_year)

# Get most recent household size per country
hh_latest = excel2_df[excel2_df[hh_size_col].notna()].sort_values('Year').drop_duplicates(subset='ISO Code', keep='last')

# Convert numeric ISO codes to 3-letter ISO codes using country_converter
try:
    # Convert numeric ISO codes to ISO3 (3-letter codes)
    # Try UNcode first (UN numeric codes follow ISO-numeric closely)
    # If that doesn't work well, country_converter will auto-detect
    iso_numeric_codes = hh_latest['ISO Code'].astype(int).astype(str).tolist()
    iso3_codes = cc.convert(names=iso_numeric_codes, src='UNcode', to='ISO3', not_found=None)
    
    # Handle case where convert returns a list or Series
    if isinstance(iso3_codes, (list, pd.Series)):
        hh_latest['Code'] = iso3_codes
    else:
        # If it's a single value (shouldn't happen but handle it)
        hh_latest['Code'] = [iso3_codes] * len(hh_latest)
    
    # Filter out rows where conversion failed (Code is None, 'not found', or invalid)
    hh_latest_clean = hh_latest[
        hh_latest['Code'].notna() & 
        (hh_latest['Code'] != 'not found') &
        (hh_latest['Code'].str.len() == 3)  # ISO3 codes are exactly 3 characters
    ].copy()
    
    if len(hh_latest_clean) > 0:
        # Merge household size data using ISO3 codes (same as other datasets)
        correlation_df = correlation_df.merge(
            hh_latest_clean[['Code', hh_size_col]], 
            on='Code', how='left'
        )
        
        # Report matching statistics
        matched_count = correlation_df[hh_size_col].notna().sum()
        total_count = len(correlation_df)
        print(f"\nHousehold size data merged: {matched_count}/{total_count} countries matched using ISO3 codes")
        
        if matched_count < total_count:
            unmatched = correlation_df[correlation_df[hh_size_col].isna()]['Code'].tolist()
            print(f"  Unmatched countries (sample): {unmatched[:10]}")
    else:
        print("Note: No household size data could be converted to ISO3 codes")
        raise ValueError("Conversion failed")
        
except Exception as e:
    print(f"Note: Could not merge household size data using country_converter: {e}")
    print("  Attempting fallback to name-based matching...")
    # Fallback to name-based matching
    try:
        if 'Entity' in gdp_df.columns:
            country_map = gdp_df[gdp_df['Year'] == correlation_year][['Code', 'Entity']].drop_duplicates()
            correlation_df_temp = correlation_df.merge(country_map, on='Code', how='left')
            
            hh_latest_clean = hh_latest[['Country or area', hh_size_col]].copy()
            correlation_df_temp = correlation_df_temp.merge(
                hh_latest_clean, 
                left_on='Entity', right_on='Country or area', how='left'
            )
            
            correlation_df = correlation_df_temp.drop(columns=['Entity', 'Country or area'], errors='ignore')
            matched_count = correlation_df[hh_size_col].notna().sum()
            total_count = len(correlation_df)
            print(f"  Fallback: Used name-based matching - {matched_count}/{total_count} countries matched")
    except Exception as e2:
        print(f"  Fallback also failed: {e2}")

print(f"\nMerged dataset for correlation analysis:")
print(f"Shape: {correlation_df.shape}")
print(f"Countries: {len(correlation_df)}")
print(f"\nColumns: {correlation_df.columns.tolist()}")
print(f"\nMissing values:")
print(correlation_df.isnull().sum())
print(f"\nFirst few rows:")
print(correlation_df.head())


In [ ]:
# Calculate Spearman correlations with measles incidence rate (including p-values)
# Using Spearman rank correlation as it's more robust to outliers
# (few countries may dominate cases/incidence)
from scipy.stats import spearmanr

# Prepare metrics for correlation
metrics = {
    'GDP per Capita': gdp_col,
    'Health Expenditure (% GDP)': health_col,
    'Urban Population (%)': urban_col,
}

if vaccine_col in correlation_df.columns:
    metrics['Vaccine Disagreement (%)'] = vaccine_col

if hh_size_col in correlation_df.columns:
    metrics['Household Size'] = hh_size_col

# Calculate correlations with incidence rate only (including p-values)
# Note: Using incidence rate per 1M population instead of total cases
correlations_incidence = {}
pvalues_incidence = {}
sample_sizes_incidence = {}

# Remove rows with missing values for each metric
for metric_name, metric_col in metrics.items():
    if metric_col in correlation_df.columns:
        # Correlation with incidence rate (Spearman rank correlation)
        data_subset = correlation_df[[incidence_col, metric_col]].dropna()
        if len(data_subset) > 2:  # Need at least 3 points for correlation
            corr_result = spearmanr(data_subset[incidence_col], data_subset[metric_col])
            correlations_incidence[metric_name] = corr_result.statistic
            pvalues_incidence[metric_name] = corr_result.pvalue
            sample_sizes_incidence[metric_name] = len(data_subset)

# Helper function to format significance
def format_significance(pval):
    if pd.isna(pval):
        return ''
    elif pval < 0.001:
        return '***'
    elif pval < 0.01:
        return '**'
    elif pval < 0.05:
        return '*'
    elif pval < 0.1:
        return '.'
    else:
        return 'ns'

# Create correlation dataframe with p-values (incidence rate only)
corr_results = pd.DataFrame({
    'Metric': list(correlations_incidence.keys()),
    'Correlation': [correlations_incidence.get(m, np.nan) for m in correlations_incidence.keys()],
    'P-value': [pvalues_incidence.get(m, np.nan) for m in correlations_incidence.keys()],
    'Significance': [format_significance(pvalues_incidence.get(m, np.nan)) for m in correlations_incidence.keys()],
    'N': [sample_sizes_incidence.get(m, np.nan) for m in correlations_incidence.keys()]
})

# Sort by absolute correlation value
corr_results['Abs_Correlation'] = corr_results['Correlation'].abs()
corr_results = corr_results.sort_values('Abs_Correlation', ascending=False).drop(columns=['Abs_Correlation'])

print("="*80)
print("SPEARMAN RANK CORRELATIONS: METRICS vs MEASLES INCIDENCE RATE")
print("="*80)
print(f"\nYear: {correlation_year}")
print(f"Number of countries analyzed: {len(correlation_df)}")
print(f"\nSignificance levels: *** p<0.001, ** p<0.01, * p<0.05, . p<0.1, ns = not significant")
print(f"\nCorrelation Results:")
print(corr_results.to_string(index=False))
print("\n" + "="*80)


In [ ]:
# Visualize correlations with p-values (incidence rate only)
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

# Bar plot of correlations with incidence rate (including significance indicators)
colors = ['red' if x < 0 else 'blue' for x in corr_results['Correlation']]
bars = ax.barh(corr_results['Metric'], corr_results['Correlation'], color=colors, alpha=0.7)
ax.axvline(x=0, color='black', linestyle='--', linewidth=1)
ax.set_xlabel('Spearman Rank Correlation Coefficient', fontsize=12)
ax.set_title(f'Spearman Correlation with Measles Incidence Rate\n(Year {correlation_year})', fontsize=14, fontweight='bold')
ax.set_xlim(-1, 1)
ax.grid(True, alpha=0.3, axis='x')
ax.invert_yaxis()

# Add significance indicators to bars
for i, (bar, sig) in enumerate(zip(bars, corr_results['Significance'])):
    if sig and sig != 'ns':
        x_pos = bar.get_width()
        if x_pos < 0:
            x_pos = x_pos - 0.05
        else:
            x_pos = x_pos + 0.05
        ax.text(x_pos, bar.get_y() + bar.get_height()/2, sig, 
                va='center', fontsize=10, fontweight='bold')

# Add legend for significance levels
fig.text(0.5, 0.02, 'Significance: *** p<0.001, ** p<0.01, * p<0.05, . p<0.1', 
         ha='center', fontsize=10, style='italic')

plt.tight_layout()
plt.subplots_adjust(bottom=0.1)
plt.show()


In [ ]:
# Create scatter plots showing relationships with p-values
# Define format_significance function if not already defined
if 'format_significance' not in globals():
    def format_significance(pval):
        if pd.isna(pval):
            return ''
        elif pval < 0.001:
            return '***'
        elif pval < 0.01:
            return '**'
        elif pval < 0.05:
            return '*'
        elif pval < 0.1:
            return '.'
        else:
            return 'ns'

# Determine number of metrics to plot
n_metrics = len([m for m in metrics.values() if m in correlation_df.columns])
n_cols = min(3, n_metrics)
n_rows = (n_metrics + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(6*n_cols, 5*n_rows))
if n_metrics == 1:
    axes = [axes]
elif n_rows == 1:
    axes = axes if isinstance(axes, np.ndarray) else [axes]
else:
    axes = axes.flatten()

plot_idx = 0
for metric_name, metric_col in metrics.items():
    if metric_col in correlation_df.columns and plot_idx < len(axes):
        # Remove missing values - using incidence rate instead of total cases
        plot_data = correlation_df[[incidence_col, metric_col]].dropna()
        
        if len(plot_data) > 2:
            ax = axes[plot_idx]
            ax.scatter(plot_data[metric_col], plot_data[incidence_col], alpha=0.6, s=50, color='crimson')
            ax.set_xlabel(metric_name, fontsize=11)
            ax.set_ylabel('Measles Incidence Rate (per 1M)', fontsize=11)
            # Format title with correlation and p-value
            corr_val = correlations_incidence.get(metric_name, np.nan)
            pval = pvalues_incidence.get(metric_name, np.nan)
            sig = format_significance(pval)
            if not pd.isna(pval):
                title = f'{metric_name} vs Measles Incidence Rate\n(ρ={corr_val:.3f}, p={pval:.4f}{sig})'
            else:
                title = f'{metric_name} vs Measles Incidence Rate\n(ρ={corr_val:.3f})'
            ax.set_title(title, fontsize=12, fontweight='bold')
            ax.grid(True, alpha=0.3)
            
            # Add trend line
            if len(plot_data) > 1:
                z = np.polyfit(plot_data[metric_col], plot_data[incidence_col], 1)
                p = np.poly1d(z)
                x_line = np.linspace(plot_data[metric_col].min(), plot_data[metric_col].max(), 100)
                ax.plot(x_line, p(x_line), "r--", alpha=0.8, linewidth=2)
            
            plot_idx += 1

# Hide unused subplots
for idx in range(plot_idx, len(axes)):
    axes[idx].axis('off')

plt.tight_layout()
plt.show()


In [ ]:
# Create a comprehensive correlation heatmap (incidence rate only)
# Prepare data for heatmap
heatmap_data = correlation_df[[incidence_col]].copy()

# Add metrics
for metric_name, metric_col in metrics.items():
    if metric_col in correlation_df.columns:
        heatmap_data[metric_name] = correlation_df[metric_col]

# Calculate correlation matrix using Spearman rank correlation
corr_matrix = heatmap_data.corr(method='spearman')

# Plot heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, fmt='.3f', cmap='coolwarm', center=0,
            square=True, linewidths=1, cbar_kws={"shrink": 0.8}, 
            xticklabels=True, yticklabels=True)
plt.title(f'Spearman Correlation Matrix: Measles Incidence Rate vs Metrics\n(Year {correlation_year})', 
          fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

print("\nCorrelation Matrix:")
print(corr_matrix.round(3))


In [ ]:
# Create scatter plots to visualize relationships
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Remove rows with missing values for plotting
plot_df = merged_df[numeric_cols].dropna()

# GDP vs Health Expenditure
axes[0, 0].scatter(plot_df[gdp_col], plot_df[health_col], alpha=0.6, s=50)
axes[0, 0].set_xlabel('GDP per Capita')
axes[0, 0].set_ylabel('Health Expenditure (% of GDP)')
axes[0, 0].set_title('GDP per Capita vs Health Expenditure', fontsize=14, fontweight='bold')
axes[0, 0].grid(True, alpha=0.3)
# Add trend line
z = np.polyfit(plot_df[gdp_col], plot_df[health_col], 1)
p = np.poly1d(z)
axes[0, 0].plot(plot_df[gdp_col], p(plot_df[gdp_col]), "r--", alpha=0.8)

# GDP vs Urban Population
axes[0, 1].scatter(plot_df[gdp_col], plot_df[urban_col], alpha=0.6, s=50, color='green')
axes[0, 1].set_xlabel('GDP per Capita')
axes[0, 1].set_ylabel('Urban Population (%)')
axes[0, 1].set_title('GDP per Capita vs Urban Population', fontsize=14, fontweight='bold')
axes[0, 1].grid(True, alpha=0.3)
z = np.polyfit(plot_df[gdp_col], plot_df[urban_col], 1)
p = np.poly1d(z)
axes[0, 1].plot(plot_df[gdp_col], p(plot_df[gdp_col]), "r--", alpha=0.8)

# Health Expenditure vs Vaccine Disagreement
axes[1, 0].scatter(plot_df[health_col], plot_df[vaccine_col], alpha=0.6, s=50, color='orange')
axes[1, 0].set_xlabel('Health Expenditure (% of GDP)')
axes[1, 0].set_ylabel('Vaccine Disagreement (%)')
axes[1, 0].set_title('Health Expenditure vs Vaccine Disagreement', fontsize=14, fontweight='bold')
axes[1, 0].grid(True, alpha=0.3)
z = np.polyfit(plot_df[health_col], plot_df[vaccine_col], 1)
p = np.poly1d(z)
axes[1, 0].plot(plot_df[health_col], p(plot_df[health_col]), "r--", alpha=0.8)

# Urban Population vs Vaccine Disagreement
axes[1, 1].scatter(plot_df[urban_col], plot_df[vaccine_col], alpha=0.6, s=50, color='purple')
axes[1, 1].set_xlabel('Urban Population (%)')
axes[1, 1].set_ylabel('Vaccine Disagreement (%)')
axes[1, 1].set_title('Urban Population vs Vaccine Disagreement', fontsize=14, fontweight='bold')
axes[1, 1].grid(True, alpha=0.3)
z = np.polyfit(plot_df[urban_col], plot_df[vaccine_col], 1)
p = np.poly1d(z)
axes[1, 1].plot(plot_df[urban_col], p(plot_df[urban_col]), "r--", alpha=0.8)

plt.tight_layout()
plt.show()


## 13. Summary Statistics


In [ ]:
# Generate comprehensive summary statistics
print("="*80)
print("COMPREHENSIVE SUMMARY STATISTICS")
print("="*80)

for name, df in datasets.items():
    print(f"\n{name}:")
    print("-" * 80)
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    if numeric_cols:
        summary = df[numeric_cols].describe()
        print(summary)
        
        # Additional statistics
        for col in numeric_cols:
            if col != 'Year' and col != 'Code':
                print(f"\n{col}:")
                print(f"  Skewness: {df[col].skew():.4f}")
                print(f"  Kurtosis: {df[col].kurtosis():.4f}")
                print(f"  Median: {df[col].median():.4f}")
                print(f"  IQR: {df[col].quantile(0.75) - df[col].quantile(0.25):.4f}")


## 14. Key Findings and Insights


In [ ]:
# Generate key insights
print("="*80)
print("KEY FINDINGS AND INSIGHTS")
print("="*80)

# GDP Insights
latest_gdp_data = gdp_df[gdp_df['Year'] == gdp_df['Year'].max()]
print(f"\n1. GDP PER CAPITA:")
print(f"   - Latest year: {gdp_df['Year'].max()}")
print(f"   - Countries with data: {latest_gdp_data['Entity'].nunique()}")
print(f"   - Average GDP per capita: ${latest_gdp_data[gdp_col].mean():,.2f}")
print(f"   - Median GDP per capita: ${latest_gdp_data[gdp_col].median():,.2f}")
print(f"   - Highest: {latest_gdp_data.loc[latest_gdp_data[gdp_col].idxmax(), 'Entity']} (${latest_gdp_data[gdp_col].max():,.2f})")
print(f"   - Lowest: {latest_gdp_data.loc[latest_gdp_data[gdp_col].idxmin(), 'Entity']} (${latest_gdp_data[gdp_col].min():,.2f})")

# Health Expenditure Insights
latest_health_data = health_exp_df[health_exp_df['Year'] == health_exp_df['Year'].max()]
print(f"\n2. HEALTH EXPENDITURE:")
print(f"   - Latest year: {health_exp_df['Year'].max()}")
print(f"   - Countries with data: {latest_health_data['Entity'].nunique()}")
print(f"   - Average health expenditure: {latest_health_data[health_col].mean():.2f}% of GDP")
print(f"   - Median health expenditure: {latest_health_data[health_col].median():.2f}% of GDP")
print(f"   - Highest: {latest_health_data.loc[latest_health_data[health_col].idxmax(), 'Entity']} ({latest_health_data[health_col].max():.2f}%)")
print(f"   - Lowest: {latest_health_data.loc[latest_health_data[health_col].idxmin(), 'Entity']} ({latest_health_data[health_col].min():.2f}%)")

# Vaccine Insights
latest_vaccine_data = vaccine_df[vaccine_df['Year'] == vaccine_df['Year'].max()]
print(f"\n3. VACCINE DISAGREEMENT:")
print(f"   - Latest year: {vaccine_df['Year'].max()}")
print(f"   - Countries with data: {latest_vaccine_data['Entity'].nunique()}")
print(f"   - Average disagreement: {latest_vaccine_data[vaccine_col].mean():.2f}%")
print(f"   - Median disagreement: {latest_vaccine_data[vaccine_col].median():.2f}%")
print(f"   - Highest: {latest_vaccine_data.loc[latest_vaccine_data[vaccine_col].idxmax(), 'Entity']} ({latest_vaccine_data[vaccine_col].max():.2f}%)")
print(f"   - Lowest: {latest_vaccine_data.loc[latest_vaccine_data[vaccine_col].idxmin(), 'Entity']} ({latest_vaccine_data[vaccine_col].min():.2f}%)")

# Urban Population Insights
latest_urban_data = urban_df[urban_df['Year'] == urban_df['Year'].max()]
print(f"\n4. URBAN POPULATION:")
print(f"   - Latest year: {urban_df['Year'].max()}")
print(f"   - Countries with data: {latest_urban_data['Entity'].nunique()}")
print(f"   - Average urban population: {latest_urban_data[urban_col].mean():.2f}%")
print(f"   - Median urban population: {latest_urban_data[urban_col].median():.2f}%")
print(f"   - Most urbanized: {latest_urban_data.loc[latest_urban_data[urban_col].idxmax(), 'Entity']} ({latest_urban_data[urban_col].max():.2f}%)")
print(f"   - Least urbanized: {latest_urban_data.loc[latest_urban_data[urban_col].idxmin(), 'Entity']} ({latest_urban_data[urban_col].min():.2f}%)")

# Relationship Insights
if len(plot_df) > 0:
    print(f"\n5. RELATIONSHIPS (Year {analysis_year}, {len(plot_df)} countries):")
    print(f"   - GDP vs Health Expenditure correlation: {plot_df[gdp_col].corr(plot_df[health_col]):.3f}")
    print(f"   - GDP vs Urban Population correlation: {plot_df[gdp_col].corr(plot_df[urban_col]):.3f}")
    print(f"   - Health Expenditure vs Vaccine Disagreement correlation: {plot_df[health_col].corr(plot_df[vaccine_col]):.3f}")
    print(f"   - Urban Population vs Vaccine Disagreement correlation: {plot_df[urban_col].corr(plot_df[vaccine_col]):.3f}")
